In [ ]:
import os
import pandas as pd
import numpy as np
import torch
from sklearn.preprocessing import StandardScaler

In [ ]:
data = pd.read_csv("/content/abalone.data - abalone.data.csv")

FileNotFoundError: [Errno 2] No such file or directory: '/content/abalone.data - abalone.data.csv'

In [ ]:
print(data.head())

In [ ]:
data = pd.get_dummies(data, columns=['Sex'])

In [ ]:
continuous_features = ['Length',  'Diameter' , 'Height' , 'Whole_weight',  'Shucked_weight',  'Viscera_weight', 'Shell_weight']
scaler = StandardScaler()
data[continuous_features] = scaler.fit_transform(data[continuous_features])

In [ ]:
data = data.astype(float)
print(data.head())

In [ ]:
import torch.nn as nn
from sklearn.model_selection import train_test_split

In [ ]:
X = data.drop(columns=['Rings'])
y = data['Rings']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
import torch

In [ ]:
X_train_tensor = torch.tensor(X_train.values, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test.values, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test.values, dtype=torch.float32)

In [ ]:
import torch.nn.functional as F

class NeuralNetwork(nn.Module):
    def __init__(self, input_dim=10, hidden_dim1=64, hidden_dim2=32, output_dim=1):
        super(NeuralNetwork, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim1)
        self.fc2 = nn.Linear(hidden_dim1, hidden_dim2)
        self.fc3 = nn.Linear(hidden_dim2, output_dim)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


In [ ]:
model= NeuralNetwork(input_dim=10, hidden_dim1=64, hidden_dim2=32, output_dim=1)
print(model)
import torch.optim as optim
model = NeuralNetwork()

In [ ]:
#Mean Squared Error (MSE) loss function
criterion = nn.MSELoss()
# Stochastic Gradient Descent (SGD) optimizer with learning rate 0.1
optimizer = optim.SGD(model.parameters(), lr=0.1)

In [ ]:
num_epochs=100
def train_model(model, criterion, optimizer, X_train, y_train, num_epochs=100):
    for epoch in range(num_epochs):
      loss=[]
        # Set the model to training mode
model.train()

 # Forward pass
outputs = model(X_train_tensor)
loss = criterion(outputs, y_train_tensor)

# Backward pass and optimization
optimizer.zero_grad()
loss.backward()
optimizer.step()
for epoch in range(num_epochs):
# Print training loss every 10 epochs
  if (epoch+1) % 10 == 0:
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item()}')

def evaluate(model, criterion, X_test, y_test):
    # Set the model to evaluation mode
    model.eval()

    # Disable gradient calculation
    with torch.no_grad():
        outputs = model(X_test)
        loss = criterion(outputs, y_test)  # Ensure target has same shape as output

    return loss.item()

# Train the model
train_model(model, criterion, optimizer, X_train_tensor, y_train_tensor, num_epochs=100)

# Evaluate the model
mse = evaluate(model, criterion, X_test_tensor, y_test_tensor)
print(f'Mean Squared Error (MSE) on Testing Set: {mse}')

In [ ]:
def evaluate(model, criterion, X_test, y_test):
    # Set the model to evaluation mode
    model.eval()
learning_rates = [0.001, 0.01, 0.1]
batch_sizes = [32, 64, 128]
hidden_nodes = [32, 64, 128]
hidden_layers = [1, 2, 3]

results = {}

# Iterate over different hyperparameter combinations
for lr in learning_rates:
    for bs in batch_sizes:
        for hn in hidden_nodes:
            for hl in hidden_layers:
                # Define and train the model
                model =NeuralNetwork(input_dim=10, hidden_dim1=hn, hidden_dim2=hn, output_dim=1)
                optimizer = optim.SGD(model.parameters(), lr=lr)
                train_model(model, criterion, optimizer, X_train_tensor, y_train_tensor, num_epochs=100)

                # Evaluate the model
                mse = evaluate(model, criterion, X_test_tensor, y_test_tensor)

                # Store the results
                results[(lr, bs, hn, hl)] = mse

In [ ]:
results_sgd = {}
results_adagrad = {}

# Iterate over different hyperparameter combinations
for lr in learning_rates:
    for bs in batch_sizes:
        for hn in hidden_nodes:
            for hl in hidden_layers:
                # Define and train the model with SGD optimizer
                model_sgd =NeuralNetwork(input_dim=10, hidden_dim1=hn, hidden_dim2=hn, output_dim=1)
                optimizer_sgd = optim.SGD(model_sgd.parameters(), lr=lr)
                train_model(model_sgd, criterion, optimizer_sgd, X_train_tensor, y_train_tensor, num_epochs=100)
                mse_sgd = evaluate(model_sgd, criterion, X_test_tensor, y_test_tensor)
                results_sgd[(lr, bs, hn, hl)] = mse_sgd

                # Define and train the model with Adagrad optimizer
                model_adagrad =NeuralNetwork(input_dim=10, hidden_dim1=hn, hidden_dim2=hn, output_dim=1)
                optimizer_adagrad = optim.Adagrad(model_adagrad.parameters(), lr=lr)
                train_model(model_adagrad, criterion, optimizer_adagrad, X_train_tensor, y_train_tensor, num_epochs=100)
                mse_adagrad = evaluate(model_adagrad, criterion, X_test_tensor, y_test_tensor)
                results_adagrad[(lr, bs, hn, hl)] = mse_adagrad


In [ ]:
class NeuralNetworkWithSigmoid(nn.Module):
    def __init__(self, input_dim=10, hidden_layers=10, hidden_dim=64, output_dim=1):
        super(NeuralNetworkWithSigmoid, self).__init__()
        self.input_layer = nn.Linear(input_dim, hidden_dim)
        self.hidden_layers = nn.ModuleList([
            nn.Linear(hidden_dim, hidden_dim) for _ in range(hidden_layers)
        ])
        self.output_layer = nn.Linear(hidden_dim, output_dim)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.sigmoid(self.input_layer(x))
        for hidden_layer in self.hidden_layers:
            x = self.sigmoid(hidden_layer(x))
        x = self.output_layer(x)
        return x

# Model with 10 hidden layers
model_with_sigmoid_10_layers =NeuralNetworkWithSigmoid(hidden_layers=10)

In [ ]:
# Sigmoid function in Python
import matplotlib.pyplot as plt
import numpy as np
def sigmoid(x):
    return 1 / (1 + torch.exp(-x))
x_values = np.linspace(-5, 5, 50)
x_values_tensor = torch.tensor(x_values, dtype=torch.float32)
y_values_tensor = sigmoid(x_values_tensor)

plt.plot(x_values, y_values_tensor.numpy())
plt.title('Sigmoid Function')
plt.xlabel('Input')
plt.ylabel('Output')
plt.grid(True)
plt.show()